[homework question link](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/03-vector-search/homework.md)

# Q1. Getting the embeddings model

```bash
pip install -U sentence-transformers
```

In [2]:
model_name = 'multi-qa-distilbert-cos-v1'
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

In [4]:
user_question = "I just discovered the course. Can I still join it?"
embedding_model.encode(user_question)[0]

0.078222655

In [5]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [10]:
ml_docs = []
for doc in documents:
    if doc['course'] == 'machine-learning-zoomcamp':
        ml_docs.append(doc)

In [11]:
len(ml_docs)

375

# Q2. Creating the embeddings

In [13]:
import numpy as np

In [16]:
embeddings = []
for doc in ml_docs:
    qa_text = doc['question'] + " " + doc['text']
    embeddings.append(embedding_model.encode(qa_text))

In [17]:
X = np.array(embeddings)
X.shape

(375, 768)

# Q3. Search